In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import numpy as np

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from pythainlp import sent_tokenize, word_tokenize

In [5]:
docs = pd.read_csv('datasets/Open source [WeVis-Promise Tracker] Data - promise.csv')

In [47]:
docs

,promiseId,party,topic,promiseTitle,status,explain,isNCPO,refPicture,picturesDrive,vdo,nameLink1,urlLink1,nameLink2,urlLink2,nameLink3,urlLink3,nameLink4,urlLink4
0,1,พลังไทยรักไทย,economics,1 จังหวัด 1 แหล่งท่องเที่ยว,nodata,ซึ่งไม่พบข้อมูล 1 จังหวัด 1 แหล่งท่องเที่ยว แต...,False,พลังไทยรักไทย_2,https://drive.google.com/drive/folders/1snFv2j...,-,นโยบายพรรพลังไทยรักไทย - line today,https://today.line.me/th/v2/article/YWmgPL?ima...,ไฮไลต์ 77 จังหวัดและสถานที่ท่องเที่ยว - ไทยรัฐ,https://www.thairath.co.th/lifestyle/travel/th...,NaN,NaN,NaN,NaN
1,2,พลังไทยรักไทย,equality,1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญาตรีฟรี มีงานทำ,nodata,ซึ่งไม่พบข้อมูล1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญ...,False,พลังไทยรักไทย_2,https://drive.google.com/drive/folders/1snFv2j...,-,นโยบายพรรพลังไทยรักไทย - line today,https://today.line.me/th/v2/article/YWmgPL?ima...,"อว. เดินหน้าโครงการ ""อว. ลดค่าเทอม"" - กระทรวงก...",https://www.mhesi.go.th/index.php/news-and-ann...,NaN,NaN,NaN,NaN
2,3,พลังไทยรักไทย,economics,1 ตำบล 1 ธนาคาร,nodata,ซึ่งไม่พบข้อมูล 1 ตำบล 1 ธนาคาร,False,พลังไทยรักไทย_2,https://drive.google.com/drive/folders/1snFv2j...,-,นโยบายพรรพลังไทยรักไทย - line today,https://today.line.me/th/v2/article/YWmgPL?ima...,NaN,NaN,NaN,NaN,NaN,NaN
3,4,พลังไทยรักไทย,equality,30 บาทรักษาทุกโรคได้จริง,nodata,ซึ่งไม่พบข้อมูลเกี่ยวกับ 30 บาทรักษาทุกโรคได้จ...,False,พลังไทยรักไทย_2,https://drive.google.com/drive/folders/1snFv2j...,-,นโยบายพรรพลังไทยรักไทย - line today,https://today.line.me/th/v2/article/YWmgPL?ima...,จุดเริ่มต้นของโครงการ 30 บาทรักษาทุกโรค,https://www.nhso.go.th/page/history,สปสช. เห็นชอบให้มีการ “ยกระดับระบบหลักประกันสุ...,https://www.nhso.go.th/news/3368,นายศุภชัย ใจสมุทร ส.ส.บัญชีรายชื่อและนายทะเบีย...,https://www.facebook.com/suphachai.jaismut/pos...
4,5,พลังปวงชนไทย,equality,30 รักษาทั่วไทย,nodata,30 บาทรักษาทุกโรคเป็นนโยบายของพรรคไทยรักไทยตั้...,False,พลังปวงชนไทย_1,https://drive.google.com/drive/folders/1snFv2j...,-,จุดเริ่มต้นของโครงการ 30 บาทรักษาทุกโรค,https://www.nhso.go.th/page/history,สปสช. เห็นชอบให้มีการ “ยกระดับระบบหลักประกันสุ...,https://www.nhso.go.th/news/3368,นายศุภชัย ใจสมุทร ส.ส.บัญชีรายชื่อและนายทะเบีย...,https://www.facebook.com/suphachai.jaismut/pos...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,351,ชาติไทยพัฒนา,equality,ให้ลูกหลานเกษตรกรเรียนฟรีถึงปริญญาตรี (หากทำงา...,working,กรมส่งเสริมสหกรณ์ได้จัดทำโครงการนำลูกหลานเกษตร...,False,ชาติไทยพัฒนา_4,https://drive.google.com/drive/folders/1snFv2j...,-,โครงการนำลูกหลานเกษตรกรกลับบ้าน สานต่ออาชีพการ...,https://drive.google.com/file/d/1BLEXX13lot6_D...,รายละเอียดโครงการนำลูกหลานเกษตรกรฯ - กรมส่งเสร...,https://web.cpd.go.th/nakhonphanom/images/PDF_...,NaN,NaN,NaN,NaN
325,352,ชาติพัฒนา,equality,ให้สิทธิ์ในการนำค่ารักษาพยาบาลของผู้สูงอายุหรื...,nodata,ให้สิทธิ์ในการนำค่ารักษาพยาบาลของผู้สูงอายุหรื...,False,ชาติพัฒนา_2,https://drive.google.com/drive/folders/1snFv2j...,-,ชาติพัฒนาเป็นทางเลือก ในการก้าวข้ามความขัดแย้ง...,https://www.bangkokbiznews.com/advertorials/ne...,ชาติพัฒนาย้ำเสถียรภาพรัฐบาล-สภา แก้ปัญหาการเมื...,https://www.sanook.com/news/8496570/,ผู้มีเงินได้มีสิทธิหักลดหย่อนอะไรได้บ้าง?,https://www.rd.go.th/557.html,NaN,NaN
326,353,พลังประชารัฐ,equality,ออกเอกสารสิทธิในที่ดินกรรมสิทธิ์ พิสูจน์สิทธิ์...,working,นโยบายนี้ต้องการแก้ปัญหาการถือครองที่ดินของนัก...,False,พลังประชารัฐ_9,https://drive.google.com/drive/folders/1snFv2j...,-,‘คทช.’ แต่งตั้งกรรมการพิสูจน์สิทธิที่ดิน - มติชน,https://www.matichon.co.th/politics/news_2647539,การประชุมคณะกรรมการนโยบายที่ดินแห่งชาติ (คทช.)...,https://www.onep.go.th/%E0%B8%81%E0%B8%B2%E0%B...,สรุปผลการประชุมคณะรัฐมนตรี วันที่ 21 ธันวาคม ...,https://www.ryt9.com/s/cabt/3283622,NaN,NaN
327,354,พลังไทยรักไทย,environmental,อีสานเขียวภาค 2 เมกะโปรเจค,nodata,ซึ่งไม่พบความเคลื่อนไหวเกี่ยวกับอีสานเขียวภาค ...,False,พลังไทยรักไทย_2,https://drive.google.com/drive/folders/1snFv2j...,-,นโยบายพรรพลังไทยรักไทย - line today,https://today.line.me/th/v2/article/YWmgPL?ima...,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
vectorizer = TfidfVectorizer(tokenizer=word_tokenize)
doc_matrix = vectorizer.fit_transform(docs['promiseTitle'].values)

In [31]:
doc_matrix[0]

<1x1062 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [9]:
cosine_similarity(doc_matrix[0],doc_matrix[3])

array([[0.03962221]])

In [78]:
group_word = []
for i,base in enumerate(doc_matrix):
    mk = []
    for j,target in enumerate(doc_matrix):
        if cosine_similarity(base,target)[0][0] >= 0.35:
            if docs['promiseTitle'].values[j] == docs['promiseTitle'].values[i]:
                mk.append(docs['promiseTitle'].values[i])
                continue
            else :
                mk.append(docs['promiseTitle'].values[j])
                mk.append(docs['promiseTitle'].values[i])
    if len(mk) != 0:
        group_word.append(mk)

In [79]:
group_word

[['1 จังหวัด 1 แหล่งท่องเที่ยว',
  '1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญาตรีฟรี มีงานทำ',
  '1 จังหวัด 1 แหล่งท่องเที่ยว',
  '1 ตำบล 1 ธนาคาร',
  '1 จังหวัด 1 แหล่งท่องเที่ยว',
  'นโยบาย 1 ตำบล 1 นักศึกษาแพทย์',
  '1 จังหวัด 1 แหล่งท่องเที่ยว',
  'เพิ่มนักโภชนาการ 1 คน ต่อ 1 เขต การศึกษา',
  '1 จังหวัด 1 แหล่งท่องเที่ยว',
  'สร้าง co-working space ใน กทม. 1 แขวง 1 ที่ ไม่คิดค่าใช้จ่าย ไวไฟฟรี',
  '1 จังหวัด 1 แหล่งท่องเที่ยว'],
 ['1 จังหวัด 1 แหล่งท่องเที่ยว',
  '1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญาตรีฟรี มีงานทำ',
  '1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญาตรีฟรี มีงานทำ',
  '1 ตำบล 1 ธนาคาร',
  '1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญาตรีฟรี มีงานทำ',
  'นโยบาย 1 ตำบล 1 นักศึกษาแพทย์',
  '1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญาตรีฟรี มีงานทำ',
  'เพิ่มนักโภชนาการ 1 คน ต่อ 1 เขต การศึกษา',
  '1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญาตรีฟรี มีงานทำ',
  'รักษาฟรีทุกโรค แบบมีคุณภาพ พร้อมรถตู้พยาบาลประจำตำบล ตำบลละ 1 คัน',
  '1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญาตรีฟรี มีงานทำ',
  'สร้าง co-working space ใน กทม. 1 แขวง 1 ที่ ไม่คิดค่

In [11]:
def check_similar(list1,list2):
    count = 0
    for i in list1:
        if i in list2:
            count += 1
    if (count/len(list2))*100 >= 80:
        return True
    else:
        return False

In [12]:
check_similar(group_word[1],group_word[0])

True

# Kmeans

In [32]:
kmeans = KMeans(n_clusters=24)
clustered_docs = kmeans.fit_predict(doc_matrix)

In [33]:
clustered_docs

array([ 8,  8,  8,  1,  3,  0,  4,  4,  4,  9,  0, 20,  4,  1, 20, 10,  9,
       14,  4, 18, 12, 16,  0, 18, 11, 10, 20, 15,  1, 12, 11,  9, 18, 16,
       16, 19, 18, 23, 16, 13, 16, 13, 13, 15, 13,  3, 17,  2,  4,  6, 15,
       10,  9,  7,  2,  4,  8, 20,  1,  9, 23,  9,  1,  9,  9, 23, 16,  0,
        0, 17, 10,  1, 10, 15,  5, 12,  6,  0, 14,  1,  4, 12,  4, 14,  8,
        4, 16,  1,  6, 15, 20, 20, 20,  2, 20, 13,  0,  6, 12,  6, 12,  9,
       23, 23,  4,  2,  0,  2,  6,  6, 17,  8,  0, 20, 14, 14, 14, 16,  4,
       14,  5, 16,  4,  4,  1,  4, 13,  7,  7,  7, 21, 18,  5, 12,  3, 19,
        4,  2,  2,  9,  8,  1,  2,  4,  6, 16,  6,  4,  3, 16, 19,  2, 16,
       20, 20, 12, 19, 12,  4, 11, 22,  1, 13, 16, 12,  9,  4, 14, 22,  4,
       11,  4,  5, 17, 15, 19, 16, 16, 16,  4, 22, 15, 23, 16,  0,  2,  2,
        5, 15,  9,  8,  2,  7,  7,  7,  7, 20, 23, 21, 23,  9,  2, 17,  4,
       11,  2, 10, 15, 15,  4,  6, 10, 23,  4,  4, 15,  8,  4, 11, 11, 11,
       15, 15, 22, 15, 11

In [48]:
newdf = pd.DataFrame(columns=['data','party','label_clusters'],data=np.array([docs['promiseTitle'].values,docs['party'].values,clustered_docs]).T)

In [49]:
all_df = newdf[newdf['label_clusters'] == 0]
for i in range(1,24):
  all_df = all_df.append(newdf[newdf['label_clusters'] == i])

/var/folders/lb/rzz41thx5dlg4slgfnphv6n00000gn/T/ipykernel_3951/3122857248.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_df = all_df.append(newdf[newdf['label_clusters'] == i])
/var/folders/lb/rzz41thx5dlg4slgfnphv6n00000gn/T/ipykernel_3951/3122857248.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_df = all_df.append(newdf[newdf['label_clusters'] == i])
/var/folders/lb/rzz41thx5dlg4slgfnphv6n00000gn/T/ipykernel_3951/3122857248.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_df = all_df.append(newdf[newdf['label_clusters'] == i])
/var/folders/lb/rzz41thx5dlg4slgfnphv6n00000gn/T/ipykernel_3951/3122857248.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a f

In [52]:
all_df

,data,party,label_clusters
5,LGBT สิทธิต้องเท่าเทียม\n\n,ประชาภิวัฒน์,0
10,กระจายอำนาจอธิปไตยให้ชุมชนท้องถิ่น,ประชาชาติ,0
22,เกษตรกรมีที่ดินทำกินและเป็นเจ้าของกรรมสิทธิ์ใน...,ประชาชาติ,0
67,"จัดสรรโดยนำที่ดินของรัฐ 1,000,000 ไร่ ให้เกษตรกร",พลังท้องถิ่นไท,0
68,จัดสรรที่ดินรัฐให้ประชาชนอยู่อาศัยและทำกิน,พลังปวงชนไทย,0
...,...,...,...
103,ทำทุ่งกุลาร้องไห้ 2.1 ล้านไร่ ใน 5 จังหวัด ให้...,รวมพลังประชาชาติไทย,23
182,พัฒนาสถาบันฝึกอาชีวะ 900 โรงทั่วประเทศ,ก้าวไกล/อนาคตใหม่,23
197,เพิ่มพลังองค์กรท้องถิ่น พัฒนาสังคมจากฐานราก,พลังท้องถิ่นไท,23
199,เพิ่มรากแก้วประเทศไทย สร้างองค์กรปกครองท้องถิ่...,พลังท้องถิ่นไท,23


In [76]:
all_df = newdf[newdf['label_clusters'] == 0]
for i in range(1,24):
  all_df = all_df.append(newdf[newdf['label_clusters'] == i])
mk_list = []
for i in range(1,24):
    subdict = dict()
    subdict['group'] = str(i)
    party = all_df[all_df['label_clusters'] == i][['party']].values
    title = all_df[all_df['label_clusters'] == i][['data']].values
    policy = []
    for i in range(len(party)):
        policy_dict = dict()
        policy_dict['party'] = party[i][0]
        policy_dict['title'] = title[i][0]
        policy.append(policy_dict)
    subdict['policy'] = policy
    mk_list.append(subdict)

In [77]:
mk_list

[{'group': '1',
  'policy': [{'party': 'พลังไทยรักไทย', 'title': '30 บาทรักษาทุกโรคได้จริง'},
   {'party': 'ภูมิใจไทย',
    'title': 'กัญชาไทย ปลูกได้เสรี และนำไปใช้ทางการแพทย์'},
   {'party': 'ประชาชนปฏิรูป',
    'title': 'แก้ไขปัญหาทุกข์ร้อนให้ประชาชนด้วยกฎหมายจนถึงที่สุด'},
   {'party': 'ประชาชนปฏิรูป',
    'title': 'จัดตั้ง "สภาประชาชนปฏิรูปจังหวัด" ทั้ง 77 จังหวัด'},
   {'party': 'ประชาธิปไตยใหม่', 'title': 'จัดตั้งกองทุนบำนาญประชาชน'},
   {'party': 'พลังท้องถิ่นไท',
    'title': 'จัดให้ทุกชุมชนและท้องถิ่นได้เรียนฟรีหลายภาษา'},
   {'party': 'ก้าวไกล/อนาคตใหม่',
    'title': 'เชื่อมโยงข้อมูลรัฐกับประชาชน ด้วย Single Digital ID'},
   {'party': 'ประชาธิปัตย์', 'title': 'เด็กทุกคนพูดภาษาอังกฤษได้'},
   {'party': 'ครูไทยเพื่อประชาชน',
    'title': 'บัตรคนดีปลดหนี้ประชาชน ทำดีได้เงิน'},
   {'party': 'พลังท้องถิ่นไท',
    'title': 'ประชาชนใหญ่กว่านักการเมือง ให้ประชาชนมีอำนาจตรวจสอบ'},
   {'party': 'ไทยศรีวิไลย์',
    'title': 'เปลี่ยนแปลงระบบยุติธรรมให้เป็นธรรมกับประชาชนทุกฐานะ'},
   {'

In [20]:
all_df.to_csv('mk.csv',index=False)